In [1]:
# imports
import torch
import tensorflow as tf
import numpy as np
import csv
from collections import defaultdict

from transformers import (TFAutoModelWithLMHead, AutoTokenizer, 
    TFTrainer, TFTrainingArguments, TFT5ForConditionalGeneration, T5Config)

from mockup import Argument
from transformers import AutoTokenizer, AutoModelWithLMHead


2021-08-23 19:21:22.156583: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-23 19:21:22.156604: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# this is taken from 
# https://colab.research.google.com/github/snapthat/TF-T5-text-to-text/blob/master/snapthatT5/notebooks/TF-T5-Datasets%20Training.ipynb#scrollTo=2xcGqd9qDXOF
class identFrameT5(TFT5ForConditionalGeneration):
    def __init__(self, *args, log_dir=None, cache_dir= None, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_tracker= tf.keras.metrics.Mean(name='loss') 
    
    @tf.function
    def train_step(self, data):
        print("HERE!")
        print(type(data))
        print("0:", data[0])
        x = data[0]
        y = x["labels"]
        y = tf.reshape(y, [-1, 1])
        with tf.GradientTape() as tape:
            outputs = self(x, training=True)
            loss = outputs[0]
            logits = outputs[1]
            loss = tf.reduce_mean(loss)
            
            grads = tape.gradient(loss, self.trainable_variables)
            
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        lr = self.optimizer._decayed_lr(tf.float32)
        
        self.loss_tracker.update_state(loss)        
        self.compiled_metrics.update_state(y, logits)
        metrics = {m.name: m.result() for m in self.metrics}
        metrics.update({'lr': lr})
        
        return metrics

    def test_step(self, data):
        print("HERE2!")
        print(data)
        x = data[0]
        y = x["labels"]
        y = tf.reshape(y, [-1, 1])
        output = self(x, training=False)
        loss = output[0]
        loss = tf.reduce_mean(loss)
        logits = output[1]
        
        self.loss_tracker.update_state(loss)
        self.compiled_metrics.update_state(y, logits)
        return {m.name: m.result() for m in self.metrics}

In [3]:
data_csv_filename = "data/Webis-argument-framing.csv"
file = open(data_csv_filename)
dreader = csv.DictReader(file, dialect='unix', delimiter=',', quotechar='\"')
    
args = []
topic_frame_mapping = defaultdict(lambda : set())

    
for row in dreader:
    arg = Argument(row['argument_id'], row['frame_id'], row['frame'], row['topic_id'], row['topic'], row['premise'],
                   row['stance'], row['conclusion'])
    topic_frame_mapping['']
    args.append(arg)


In [4]:
X_text = []
Y_text = []
# simplest X_text

for arg in args:
    X_text.append(arg.premise + arg.conclusion)
    Y_text.append(arg.frame)

In [5]:
print('input:', X_text[0],'\n\n','output:', Y_text[1])

input: U.S. Treasury Secretary Henry Paulson summarized the rationale for the bailout in testimony before Congress in September, 2008 - "We must...avoid a continuing series of financial institution failures and frozen credit markets that threaten American families' financial well-being, the viability of businesses both small and large, and the very health of our economy."Not passing $700b bailout risks sending economy into major recession 

 output: economics


In [6]:
# setup t5
# this uses 'PreTrainedTokenizerFast' and 't5.TFT5ForConditionalGeneration' objects.
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = identFrameT5.from_pretrained("t5-small")

2021-08-23 19:21:29.248589: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-23 19:21:29.248609: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-23 19:21:29.248624: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lucy): /proc/driver/nvidia/version does not exist
2021-08-23 19:21:29.248784: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-23 19:21:29.262893: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but thi

In [7]:
# test t5
inputs = tokenizer(X_text[0],return_tensors='tf').input_ids
labels = tokenizer(Y_text[0],return_tensors='tf').input_ids
outputs = model.generate(inputs,max_length = 1000)
tokens = tokenizer.convert_ids_to_tokens(outputs[0])
tokenizer.convert_tokens_to_string(tokens)

'<pad><extra_id_0> - "We must...avoid a continuing series of financial institution failures" not passing $700b bailout risks sending economy into major recession . not passing $700b bailout risks sending economy into major recession .</s>'

In [8]:
#outs = model(inputs,labels=labels)
#print(inputs.shape)
#print(labels.shape)
#print(outs.logits.shape)
#np.mean(outs.loss)

In [9]:
X = tokenizer(X_text,return_tensors='tf',truncation=True,padding=True).input_ids[0:10]
Y = tokenizer(Y_text,return_tensors='tf',truncation=True,padding=True).input_ids[0:10]

print(X.shape)
print(Y.shape)
model.compile('adam',loss=tf.keras.losses.SparseCategoricalCrossentropy())

model.fit(x={'input_ids':X, 'labels': Y},epochs=10);

(10, 512)
(10, 24)
Epoch 1/10


2021-08-23 19:21:36.515290: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


HERE!
<class 'tuple'>
0: {'input_ids': <tf.Tensor 'data:0' shape=(None, 512) dtype=int32>, 'labels': <tf.Tensor 'data_1:0' shape=(None, 24) dtype=int32>}
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
HERE!
<class 'tuple'>
0: {'input_ids': <tf.Tensor 'data:0' shape=(None, 512) dtype=int32>, 'labels': <tf